In [1]:
%display latex

In [2]:
import algorithm_states, common

We start with a matrix $ m $ and a model improvement $ t $.

In [3]:
dim = 6
p = 5
m0 = diagonal_matrix([-10, -10, -5, 5, 5, -5])
t0 = matrix([
    [0, 0, 2/5, -1/5, -1/5, 2/15],
    [0, 0, -3/5, -1/5, 0, 0],
    [0, 0, 2/5, -1/5, 2/5, -4/15],
    [-1/5, 6/35, 0, 0, 0, 0],
    [0, 0, 0, 0, 4/5, -1/5],
    [-6/5, 1/35, 0, 0, 0, 0]
])
(t0.transpose() * m0 * t0).determinant().factor()

2^2

Initialization of variables

In [18]:
state = algorithm_states.ModelImprovementState(m0, t0)
s = []

By Remark 4.2, we can exchange the first column with another column (if necessary) such that $ \mathop{ord}_p(t_{*1}) \leq \mathop{ord}_p(t_{*j}) $ for all $ j $.

In [19]:
(o, j) = min((common.ord_vector(p, state.improvement, j), j) for j in range(0, dim))
if o < common.ord_vector(p, state.improvement, 0):
    state.switch_columns(j, 0)

Take $ s_1 = \mathop{ord}_p(t_{*1}) $.

In [20]:
s.append(common.ord_vector(p, state.improvement, 0)); s[0]

-1

We can also exchange the first row with another row (if necessary) such that $ \mathop{ord}_p(t_{11}) = s_1 $ (and perform the same exchange on these rows and columns of $ m $).

In [21]:
(o, j) = min((common.ord(p, state.improvement[j, 0]), j) for j in range(0, dim))
if o < common.ord(p, state.improvement[0, 0]):
    state.switch_rows(j, 0)

(
[  5   0   0   0   0   0]  [ -1/5  6/35     0     0     0     0]
[  0 -10   0   0   0   0]  [    0     0  -3/5  -1/5     0     0]
[  0   0  -5   0   0   0]  [    0     0   2/5  -1/5   2/5 -4/15]
[  0   0   0 -10   0   0]  [    0     0  -2/5   1/5   1/5 -2/15]
[  0   0   0   0   5   0]  [    0     0     0     0   4/5  -1/5]
[  0   0   0   0   0  -5], [ -6/5  1/35     0     0     0     0]
)

Note that $ p^{-s_1} t_{11} \in \mathbb Z_p^* $, so we can divide the first column by it, to obtain $ t_{11} = p^{s_1} $.

In [22]:
state.step((identity_matrix(dim), elementary_matrix(QQ, dim, row1=0, scale=p^s[0] / state.improvement[0, 0])))

(
[  5   0   0   0   0   0]  [  1/5  6/35     0     0     0     0]
[  0 -10   0   0   0   0]  [    0     0  -3/5  -1/5     0     0]
[  0   0  -5   0   0   0]  [    0     0   2/5  -1/5   2/5 -4/15]
[  0   0   0 -10   0   0]  [    0     0  -2/5   1/5   1/5 -2/15]
[  0   0   0   0   5   0]  [    0     0     0     0   4/5  -1/5]
[  0   0   0   0   0  -5], [  6/5  1/35     0     0     0     0]
)

Since $ s_1 = \mathop{ord}_p(t_{*1}) \leq \mathop{ord}_p(t_{*2}) \leq \mathop{ord}_p(t_{12}) $, we have $ p^{-s_1} t_{12} \in \mathbb Z_p $. Therefore, we can subtract $ p^{-s_1} t_{12} $ times $ t_{11} $ from $ t_{12} $ to get $ t_{12} = 0 $.

In the same way, we can make sure that $ t_{1j} = 0 $ for all $ j \geq 2 $.

In [23]:
for i in range(1, dim):
    state.add_column(0, i, -state.improvement[0, i] / state.improvement[0, 0], f"Make entry ({0}, {i}) equal to 0")

Make entry (0, 1) equal to 0


(
[  5   0   0   0   0   0]  [  1/5     0     0     0     0     0]
[  0 -10   0   0   0   0]  [    0     0  -3/5  -1/5     0     0]
[  0   0  -5   0   0   0]  [    0     0   2/5  -1/5   2/5 -4/15]
[  0   0   0 -10   0   0]  [    0     0  -2/5   1/5   1/5 -2/15]
[  0   0   0   0   5   0]  [    0     0     0     0   4/5  -1/5]
[  0   0   0   0   0  -5], [  6/5    -1     0     0     0     0]
)

Note that by doing this, the $ \mathop{ord}_p(t_{*j}) $ can only increase, so they will stay larger than $ \mathop{ord}_p(t_{*1}) $.

We can repeat the above for the second through $ n $th columns (by induction), to make sure that for all $ i < j $, $ t_{ij} = 0 $ and for all $ i $, $ t_{ii} = p^{s_i} $.

In [24]:
for i in range(1, dim):
    (o, j) = min((common.ord_vector(p, state.improvement, j), j) for j in range(i, dim))
    if o < common.ord_vector(p, state.improvement, i):
        state.switch_columns(j, i, description = f"Make the order of column {i} minimal")
        
    s.append(common.ord_vector(p, state.improvement, i))
    
    (o, j) = min((common.ord(p, state.improvement[j, i]), j) for j in range(i, dim))
    if o < common.ord(p, state.improvement[i, i]):
        state.switch_rows(j, i, description = f"Make the order of entry ({i}, {i}) minimal")
    
    state.step((identity_matrix(dim), elementary_matrix(QQ, dim, row1=i, scale=p^s[i] / state.improvement[i, i])), description = f"Convert entry ({i}, {i}) to the form p^s")
    
    for j in range(i + 1, dim):
        state.add_column(i, j, -state.improvement[i, j] / state.improvement[i, i], f"Make entry ({i}, {j}) equal to 0")

Make the order of column 1 minimal


(
[  5   0   0   0   0   0]  [  1/5     0     0     0     0     0]
[  0 -10   0   0   0   0]  [    0  -3/5     0  -1/5     0     0]
[  0   0  -5   0   0   0]  [    0   2/5     0  -1/5   2/5 -4/15]
[  0   0   0 -10   0   0]  [    0  -2/5     0   1/5   1/5 -2/15]
[  0   0   0   0   5   0]  [    0     0     0     0   4/5  -1/5]
[  0   0   0   0   0  -5], [  6/5     0     1     0     0     0]
)

Convert entry (1, 1) to the form p^s


(
[  5   0   0   0   0   0]  [  1/5     0     0     0     0     0]
[  0 -10   0   0   0   0]  [    0   1/5     0  -1/5     0     0]
[  0   0  -5   0   0   0]  [    0 -2/15     0  -1/5   2/5 -4/15]
[  0   0   0 -10   0   0]  [    0  2/15     0   1/5   1/5 -2/15]
[  0   0   0   0   5   0]  [    0     0     0     0   4/5  -1/5]
[  0   0   0   0   0  -5], [  6/5     0     1     0     0     0]
)

Make entry (1, 3) equal to 0


...

(
[  5   0   0   0   0   0]  [  1/5     0     0     0     0     0]
[  0 -10   0   0   0   0]  [    0   1/5     0     0     0     0]
[  0   0  -5   0   0   0]  [    0 -2/15   1/5     0     0     0]
[  0   0   0 -10   0   0]  [    0  2/15  1/10     1     0     0]
[  0   0   0   0  -5   0]  [  6/5     0     0     0    -1     0]
[  0   0   0   0   0   5], [    0     0  -2/5  -4/3     0  -1/3]
)

Convert entry (4, 4) to the form p^s


(
[  5   0   0   0   0   0]  [  1/5     0     0     0     0     0]
[  0 -10   0   0   0   0]  [    0   1/5     0     0     0     0]
[  0   0  -5   0   0   0]  [    0 -2/15   1/5     0     0     0]
[  0   0   0 -10   0   0]  [    0  2/15  1/10     1     0     0]
[  0   0   0   0  -5   0]  [  6/5     0     0     0     1     0]
[  0   0   0   0   0   5], [    0     0  -2/5  -4/3     0  -1/3]
)

Convert entry (5, 5) to the form p^s


(
[  5   0   0   0   0   0]  [  1/5     0     0     0     0     0]
[  0 -10   0   0   0   0]  [    0   1/5     0     0     0     0]
[  0   0  -5   0   0   0]  [    0 -2/15   1/5     0     0     0]
[  0   0   0 -10   0   0]  [    0  2/15  1/10     1     0     0]
[  0   0   0   0  -5   0]  [  6/5     0     0     0     1     0]
[  0   0   0   0   0   5], [    0     0  -2/5  -4/3     0     1]
)

Note that $ t s^{-1} $ is a lower triangular matrix with entries in $ \mathbb Z_p $ and all $ 1 $s on the diagonal. Taking $ r^\prime = t s^{-1} $, we can trivially find a matrix $ r $ as in the first part of Lemma 4.4 with all $ 1 $s on the diagonal.

In [25]:
s_matrix = diagonal_matrix(QQ, [p^n for n in s])

In [26]:
r = (state.improvement * s_matrix.inverse()).apply_map(lambda x: ZZ(Integers(p^(-2 * s[0]))(x))); r

[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0 16  1  0  0  0]
[ 0  9 13  1  0  0]
[ 6  0  0  0  1  0]
[ 0  0 23  7  0  1]

In [28]:
new_t = state.t[0].inverse() * r * s_matrix; new_t

[    0  -9/5 -13/5    -1     0     0]
[    0   1/5     0     0     0     0]
[    0  16/5   1/5     0     0     0]
[  1/5     0     0     0     0     0]
[    0     0 -23/5    -7     0    -1]
[  6/5     0     0     0     1     0]

In [29]:
new_t.transpose() * m0 * new_t

[ -7   0   0   0  -6   0]
[  0 -84 -50 -18   0   0]
[  0 -50  38 135   0  23]
[  0 -18 135 235   0  35]
[ -6   0   0   0  -5   0]
[  0   0  23  35   0   5]

In [30]:
(new_t.transpose() * m0 * new_t).determinant()

4